# Leonardo Gracida Munoz A01379812
# Nancy L. García Jiménez A01378043
## Red neuronal detectora de senales de trafico
Link hacia dataset grande (DATASET) usada en drive:<br>
https://drive.google.com/drive/folders/1fA9DtcOlEPpT_WcOAAgDyHBUZyQvRG0R?usp=sharing<br>
Link hacia dataset pequena (dataset_red) usada en drive:<br>
https://drive.google.com/drive/folders/1-3hc9RzfkL2eOIG2YRX-q0-WGxtW_DMA?usp=sharing

Sitio original de la dataset:<br>
https://benchmark.ini.rub.de/gtsrb_news.html

Link hacia repositorio donde esta el codigo para poder clasificar imagenes:


In [ ]:
#Importamos la libreria de tensorflow deseada
!pip install tensorflow==2.6.2
#mostramos la targeta grafica a usar
!nvidia-smi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Thu Jun 16 04:11:40 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+-------------

In [ ]:
#Importamos la dataset de google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def preprocessing(img):
  """Funcion que preporcesa la imagen a usar"""
  img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
  img =cv2.equalizeHist(img)
  i#mg=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img = img/255
  return img

In [ ]:
#Esto lo corremos una vez para poder extraer el zip del dataset en drive
"""import zipfile
TMP_DATA_DIR = "/content/drive/MyDrive/dataset_red"
to_unpack = [
    ("/content/drive/MyDrive/Train_red.zip", TMP_DATA_DIR),

]
 
for file, directory in to_unpack:
    print("Unzipping {} to {}...".format(file, directory))
    with zipfile.ZipFile(file,"r") as zip_ref:
        zip_ref.extractall(directory)"""

'import zipfile\nTMP_DATA_DIR = "/content/drive/MyDrive/dataset_red"\nto_unpack = [\n    ("/content/drive/MyDrive/Train_red.zip", TMP_DATA_DIR),\n\n]\n \nfor file, directory in to_unpack:\n    print("Unzipping {} to {}...".format(file, directory))\n    with zipfile.ZipFile(file,"r") as zip_ref:\n        zip_ref.extractall(directory)'

In [ ]:
#Importamos todas las librerias necesarias a usar
import tensorflow as tf
#Buscamos el gpu a usar
tf.config.set_visible_devices([], 'GPU')
print(tf.__version__)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
import cv2
from sklearn.model_selection import train_test_split
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def myModel(inputShape,clases):
    """Funcion que crea el modelo de la red neuronal obteniendo los paramteros
    de el tamano de la imagen y el numero de clases a clasificar"""
    #Selección de hiperparamteros                                                                                                                                                                                                                                                                                                                                                                                                                          
    # Tamano de los filtros a usar en los Conv2D                                                                                                                                                                                   
    size_of_Filter=(5,5)                                                                                                                                                                                                         
    size_of_Filter2=(3,3)                                                                                                                                                                                                        
    # Tekes batches of 2x2 pixels and avg the  
    # Tamamos batches de 2x2 para disminuir la informacion obtenida del Conv2D                                                                                                                                                                                  
    size_of_pool=(2,2)                                                                                                                                                                                                           
    model = Sequential()
    model.add(Conv2D(60, size_of_Filter,input_shape=inputShape))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=-1))
    model.add(Conv2D(60, size_of_Filter))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=-1))
    model.add(MaxPooling2D(pool_size=size_of_pool))
    model.add(Conv2D(30, size_of_Filter2))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=-1))
    model.add(Conv2D(30, size_of_Filter2))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=-1))
    model.add(MaxPooling2D(pool_size=size_of_pool))
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(500))
    model.add(Activation("relu"))
    model.add(Dropout(0.4))
    model.add(Dense(clases))
    model.add(Activation("softmax"))  
    model.compile(optimizer=tf.keras.optimizers.RMSprop(),                                                                                                                                                                       
              loss=tf.keras.losses.BinaryCrossentropy(),                                                                                                                                                                         
              metrics=[tf.keras.metrics.BinaryAccuracy(),                                                                                                                                                                        
                       tf.keras.metrics.FalseNegatives()]) 
    return model
#Path de la carpeta donde se encuentra el DATASET a utilizar
path = "/content/drive/MyDrive/dataset_red/Train" #En caso de que quieras entrenar la deteccion de 6 senales diferentes
#path = "/content/drive/MyDrive/DATASETrain" #En caso de que quieras detectar 43 senales diferentes
myList = os.listdir(path)
print("Clases totales detectadas: ",len(myList))
clases = len(myList)
#Dimensiones de la imagen a usar
imageDimesions = (32,32,1)
#Creamos el modelo
model = myModel(imageDimesions,clases)
#Construimos el modelo
model.build()
#El tamano de lote a usar 
batch_size_val=16
#El numero de epochs s usar
epochs_val= 100
testRatio = 0.2 #Proporcion de en cuanto vamos a dividir el DATASET el train y test
gamma = 1     #Factor de crecimiento de aprendizaje
learningRate = gamma * 1e-3


2.6.2
Clases totales detectadas:  6
Compilando modelo...
Modelo compilado


In [ ]:
#Extraemos las imagenes de la carpeta del dataset con cv2
count = 0
images = []
classNo = []
print("Importando Imagenes: ")
for i in myList:
    imageList = os.path.join(path,i)
    for j in os.listdir(imageList):
        imgPath = os.path.join(imageList,j)
        label = i
        curImg = cv2.imread(imgPath)
        #Le cambiamos el tamano a la imagen
        curImg = cv2.resize(curImg,(imageDimesions[1],imageDimesions[0]))
        #Preprocesamos la imagen
        curImg = preprocessing(curImg)
        curImg = np.expand_dims(curImg, axis=0)
        curImg = np.transpose(curImg, (1, 2, 0))
        images.append(curImg)
        classNo.append(int(i))
    print(count, end =" ")
    count +=1

Importando Imagenes: 
0 1 2 3 4 5 

In [ ]:
print("Numero de imagenes importadas: ",len(images))
images = np.array(images)
print("Dimensiones de las imgenes extraidas: ",images.shape)
classNo = np.array(classNo)
print(tf.__version__)

Numero de imagenes importadas:  4693
Dimensiones de las imgenes extraidas:  (4693, 32, 32, 1)
2.6.2


In [ ]:
#Dividimos la dataset en una parte para train y otra parte para test
X_train, X_test, y_train, y_test = train_test_split(images, classNo, test_size=testRatio)
#Cambismos los labels a modo categorical
y_test = to_categorical(y_test, clases)
y_train = to_categorical(y_train, clases)
print("Labels en modo categorical")
print(y_train.shape)
print(y_test.shape)
classTotal = y_train.sum(axis=0)
classWeight = classTotal.max() / classTotal
print("Proporcion de cada tipo de muestra en el dateset usado:")
print(classWeight)
classWeight = {i: classWeight[i] for i in range(len(classWeight))}

Labels en modo categorical
(3754, 6)
(939, 6)
Proporcion de cada tipo de muestra en el dateset usado:
[1.2427821 4.9842105 1.2063694 2.4790576 1.        1.3764535]


In [ ]:
print("Resumen de modelo de la red neuronal:")
print(model.summary())

Resumen de modelo de la red neuronal:
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 60)        1560      
_________________________________________________________________
activation (Activation)      (None, 28, 28, 60)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 28, 28, 60)        240       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 60)        90060     
_________________________________________________________________
activation_1 (Activation)    (None, 24, 24, 60)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 24, 24, 60)        240       
_________________________________________________________________
max_pooling2d (Max

In [ ]:
#Generacion de ruido en las imagenes
dataGen = ImageDataGenerator(
    rotation_range=5,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.15,
    fill_mode="nearest")

In [ ]:
_#Entrenamos el modelo
from keras.callbacks import EarlyStopping
#Creamos un callback para evitar overfiting
monitor_val_acc = EarlyStopping(monitor = 'val_loss', patience = 3)
H=model.fit(dataGen.flow(X_train,y_train,batch_size=batch_size_val),epochs=epochs_val,validation_data=(X_test,y_test),shuffle=1,callbacks=[monitor_val_acc])
#Salvamos el modelo
model.save('/content/drive/MyDrive/OUTPUT/signals_5_super_final')

Epoch 1/100
235/235 [==============================] - 24s 99ms/step - loss: 0.2291 - binary_accuracy: 0.9216 - false_negatives: 1023.0000 - val_loss: 0.4930 - val_binary_accuracy: 0.7755 - val_false_negatives: 697.0000
Epoch 2/100
235/235 [==============================] - 24s 100ms/step - loss: 0.0811 - binary_accuracy: 0.9748 - false_negatives: 308.0000 - val_loss: 0.1156 - val_binary_accuracy: 0.9542 - val_false_negatives: 138.0000
Epoch 3/100
235/235 [==============================] - 23s 98ms/step - loss: 0.0534 - binary_accuracy: 0.9837 - false_negatives: 190.0000 - val_loss: 0.0159 - val_binary_accuracy: 0.9957 - val_false_negatives: 13.0000
Epoch 4/100
235/235 [==============================] - 23s 98ms/step - loss: 0.0405 - binary_accuracy: 0.9883 - false_negatives: 137.0000 - val_loss: 0.0570 - val_binary_accuracy: 0.9870 - val_false_negatives: 38.0000
Epoch 5/100
235/235 [==============================] - 23s 99ms/step - loss: 0.0298 - binary_accuracy: 0.9919 - false_negati